## SVM

In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [4]:
wines_df= pd.read_csv("winequality-red.csv").drop_duplicates()
wines_df["is_good"] = wines_df.apply(lambda row: 1 if row.quality > 5 else 0, axis = 1)
X=wines_df.drop(["is_good", 'quality'], axis=1)
y=wines_df["is_good"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)

### Base SVM

In [6]:
svm_base=SVC(random_state=42)
svm_base.fit(X_train, y_train)

SVC(random_state=42)

In [7]:
preds=svm_base.predict(X_test)
comparison = pd.DataFrame({'actual':y_test, 'predicted':preds})
print("Accuracy: " + str(sum(comparison["actual"] == comparison["predicted"]) / len(comparison) * 100) + "%")

Accuracy: 65.80882352941177%


Wynik surowego SVM na naszym zbiorze danych to niecałe 66%. Wykonamy teraz standaryzację naszych zmiennych, a następnie poszukamy kombinacji parametrów dających najlepsze wyniki używając narzędzia `GridSearchCV`.

### Tuning hiperparametrów

In [8]:
X=(X-X.mean())/(X.std())
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)
cpar=[]
gpar=[]
for i in range(-4, 5):
    cpar.append(10**i)
for i in range(-4, 5):
    gpar.append(10**i)
gpar.append("auto")
gpar.append("scale")

In [9]:
params = [{'C': cpar,
        'kernel': ["rbf"], # "linear", "poly"],    bardzo długi czas wykonywania
        'gamma': gpar}]
svm_tuned=SVC(random_state=42)
gs_svm=GridSearchCV(svm_tuned, param_grid=params, scoring='accuracy', cv=4, n_jobs=2)
gs_svm.fit(X_train, y_train)
gs_svm.best_params_

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [10]:
gs_svm.score(X_test, y_test)

0.7389705882352942

Po dosyć długim przetestowaniu różnych kombinacji hiperparametrów używając narzędzia GridSearchCV, najlepszy wynik uzyskaliśmy dla `{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}`. Sprawdziliśmy następnie jeszcze wartości w okolicach tych parametrów.

In [11]:
cpar=[]
gpar=[]
for i in range(1, 11):
    cpar.append(round((10**0)*2*i, 8))
for i in range(1, 11):
    gpar.append(round((10**-3)*2*i, 8))
params = [{'C': cpar,
         'kernel': ["rbf"],
         'gamma': gpar}]
svm_tuned=SVC(random_state=42)
gs_svm=GridSearchCV(svm_tuned, param_grid=params, scoring='accuracy', cv=4, n_jobs=2)
gs_svm.fit(X_train, y_train)
gs_svm.best_params_   

{'C': 10, 'gamma': 0.012, 'kernel': 'rbf'}

In [12]:
gs_svm.score(X_test, y_test)

0.7463235294117647

Udało się jeszcze trochę polepszyć wyniki naszego modelu przyjmując parametry `'C': 10, 'gamma': 0.012, 'kernel': 'rbf`, osiągając skuteczność powyżej 74%. Nie są to jednak duże różnice.

Używając SVM nie udało się siągnąć wyniku tak dobrego jak w przypadku np. xgboosta. Powodem tego jest zapewne spore nakładanie się na siebie naszych dwóch klas - w końcu granica pomiędzy nimi to różnica między subiektywną oceną 5 a 6 w 10 stopniowej skali, a SVM nie radzi sobie szczególnie dobrze z takimi problemami. Jest on też dosyć wolny - testowanie różnych konfiguracji parametrów, szczególnie przy użyciu opcji `kernel=poly`, zajmowało godziny.